In [ ]:

pwd

'd:\\Study\\Programs\\TSAI\\EAG_V1\\Assignment_7_RAG_1\\2_URL_RAG\\notebooks'

In [2]:
import os

os.chdir("../")
print(os.getcwd())

d:\Study\Programs\TSAI\EAG_V1\Assignment_7_RAG_1\2_URL_RAG


In [3]:
import sys
sys.path.append('./client')

In [4]:
from IPython.display import display, HTML, Markdown
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from client.llm_provider import default_llm #it is gpt-4o-mini
from client.embedding_provider import OpenAIEmbeddingProvider #text-embedding-3-small
from server.utils import read_yaml_file

urllib3 SSL patched successfully
httpx Client patched to disable SSL verification
httpx AsyncHTTPTransport patched to disable SSL verification
✅ SSL verification completely disabled at all levels
SSL_CERT_FILE: None
PYTHONHTTPSVERIFY: 0
OPENAI_VERIFY_SSL_CERTS: false
OPENAI_API_SKIP_VERIFY_SSL: true
✅ SSL test success: 401


In [6]:
llm = default_llm.chat_model #it is gpt-4o-mini
embedder = OpenAIEmbeddingProvider().embeddings #text-embedding-3-small


print(llm.invoke("what is the capital of France?"))
print(len(embedder.embed_query("what is the capital of France?")))

content='The capital of France is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpyL2jPDNNVIn6H6H7ghnQwSwFArF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f0bf3504-5ddb-4076-b4a3-a18d9332d830-0' usage_metadata={'input_tokens': 14, 'output_tokens': 7, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
1536


In [7]:
# read config
config = read_yaml_file("client/config.yaml")
print(config)

{'db_index_name': 'weburl_index', 'history_index_name': 'history_index', 'reset_index': False}


In [8]:
history_index_name = config["history_index_name"]
history_index_name = os.path.join(os.getcwd(), history_index_name)
print(history_index_name)

d:\Study\Programs\TSAI\EAG_V1\Assignment_7_RAG_1\2_URL_RAG\history_index


#### Next steps 
- Perception code which will be used prepare the question based on the user query and chat history
- Decision making code which will be used to decide the action based on the question and chat history
- Action code which will be used to execute the action
- Memory code which will be used to store the chat history
- Client code which will be used to coordinate the conversation between the perception, decision making, action and memory


### Perception Code

In [14]:
from pydantic import BaseModel, Field
from typing import Literal, Optional
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableSequence
import json

from memory import ConversationMemory

In [20]:
class WebContentSearch(BaseModel):
    enhanced_user_query: str = Field(
        description="The enhanced user query to be searched on the web for similar content stored in the memory"
    )
    no_of_results: int = Field(
        1,
        description="The number of results to be returned from the web search corresponding to the user query",
    )

In [21]:
system_prompt = """
You are an intelligent assistant designed to refine user queries using current input and prior context. Your goal is to construct a precise, context-aware enhanced query that reflects the user’s true intent.

Your responsibilities are:
1. Analyze the current user message to understand the core request.
2. Examine the chat history (a list of messages with sender and content) to identify:
   - Any contextually relevant information.
   - Follow-ups or references to previous discussions.
3. Determine whether the current query is:
   a. A continuation of a previous topic.
   b. A new, unrelated query.

Based on this analysis, follow the appropriate path:

**If the current message relates to previous topics:**
- Incorporate relevant prior context into the enhanced query.
- Clarify ambiguous references or pronouns using information from the chat history.
- Resolve under-specified requests by grounding them in previous conversations.

**If the message is a new topic:**
- Focus solely on the current message.
- Do not infer or inject unrelated context from chat history.

**Process for forming the enhanced user query:**
1. Carefully review the current message.
2. Analyze the chat history (most recent last) to extract relevant past content.
3. Identify if the user is building on a previous conversation.
4. Integrate any necessary clarifications, references, or details from the chat history.
5. Output a concise, context-enriched, unambiguous enhanced query.

**Inputs:**
- `chat_history`: A list of dictionaries, each with keys `sender` and `content`.
    {chat_history}
- `user_query`: The latest message from the user.
    {user_query}

**Output Format:**
{format_instructions}
"""

In [22]:
def get_perception_chain(llm) -> RunnableSequence:
    """
    Creates and returns the perception chain for extracting travel search parameters.

    Args:
        default_llm: The default LLM to use for the chain.

    Returns:
        A LangChain runnable sequence that takes a user query and chat history,
        and returns a TravelSearch object.
    """

    # Set up a parser
    parser = PydanticOutputParser(pydantic_object=WebContentSearch)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "The user query is: {user_query}"),
        ]
    )

    prompt = prompt.partial(format_instructions=parser.get_format_instructions())

    # Create and return the chain
    return prompt | llm | parser

In [23]:
conv_id = uuid.uuid4()
store = ConversationMemory(
    embedder, index_folder=history_index_name, reset_index=True
)

2025-07-05 20:56:56.978 | WARNING  | utils:check_and_reset_index:27 - reset_index is set to True. Deleting existing index at 'd:\Study\Programs\TSAI\EAG_V1\Assignment_7_RAG_1\2_URL_RAG\history_index'
2025-07-05 20:56:56.980 | SUCCESS  | utils:check_and_reset_index:32 - Successfully deleted index folder 'd:\Study\Programs\TSAI\EAG_V1\Assignment_7_RAG_1\2_URL_RAG\history_index'


In [24]:
chat_history = store.get_conversation_as_lc_messages(str(conv_id))
print(chat_history)

[]


In [31]:
user_query = "What are all MLOPS tools which should be learnt to get a job in MLOPS?"

In [32]:
perception_chain = get_perception_chain(llm)
perception_chain

ChatPromptTemplate(input_variables=['chat_history', 'user_query'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [33]:
result = perception_chain.invoke(
    {"user_query": user_query, "chat_history": chat_history}
)
print(result)

enhanced_user_query='What are the essential MLOps tools and technologies that one should learn to secure a job in the MLOps field?' no_of_results=5


In [34]:
# create the history object
messages = [
    {"sender": "human", "content": user_query},
    {"sender": "ai", "content": result.enhanced_user_query},
]
store.store_conversation(str(conv_id), messages)

In [35]:
user_query = "Give me two articles on for the same topic?"
chat_history = store.get_conversation_as_lc_messages(str(conv_id))
print(chat_history)

result = perception_chain.invoke(
    {"user_query": user_query, "chat_history": chat_history}
)
print(result)

[{'role': 'human', 'content': 'What are all MLOPS tools which should be learnt to get a job in MLOPS?'}, {'role': 'ai', 'content': 'What are the essential MLOps tools and technologies that one should learn to secure a job in the MLOps field?'}]
enhanced_user_query='Can you provide two articles on essential MLOps tools and technologies that one should learn to secure a job in the MLOps field?' no_of_results=2


In [36]:
# create the history object
messages = [
    {"sender": "human", "content": user_query},
    {"sender": "ai", "content": result.enhanced_user_query},
]
store.store_conversation(str(conv_id), messages)

In [37]:
chat_history = store.get_conversation_as_lc_messages(str(conv_id))
print(chat_history)

[{'role': 'human', 'content': 'What are all MLOPS tools which should be learnt to get a job in MLOPS?'}, {'role': 'ai', 'content': 'What are the essential MLOps tools and technologies that one should learn to secure a job in the MLOps field?'}, {'role': 'human', 'content': 'Give me two articles on for the same topic?'}, {'role': 'ai', 'content': 'Can you provide two articles on essential MLOps tools and technologies that one should learn to secure a job in the MLOps field?'}]


In [38]:
# create a function where we will pass the history object and a list of conversation and the chain
# and it will keep processing the conversation and return the enhanced user query


def process_conversation_and_enhance_query(
    history_store, conversation_list, chain, conv_id
):
    """
    Processes a list of user queries (conversation_list) using the provided chain and history store.
    For each user query, it retrieves the chat history, invokes the chain, stores the conversation,
    and returns a list of enhanced user queries.

    Args:
        history_store: The object responsible for storing and retrieving conversation history.
        conversation_list: List of user queries (strings) to process.
        chain: The chain object with an 'invoke' method.
        conv_id: The conversation id to store the conversation.

    Returns:
        List of enhanced user queries (one for each user query in conversation_list).
    """
    enhanced_queries = []
    for user_query in conversation_list:
        # Retrieve chat history for the conversation
        chat_history = history_store.get_conversation_as_lc_messages(str(conv_id))
        # Invoke the chain to get the result
        result = chain.invoke({"user_query": user_query, "chat_history": chat_history})
        # Store the conversation
        messages = [
            {"sender": "human", "content": user_query},
            {"sender": "ai", "content": result.enhanced_user_query},
        ]
        history_store.store_conversation(str(conv_id), messages)
        # Collect the enhanced user query
        enhanced_queries.append(result.enhanced_user_query)
    return enhanced_queries

In [40]:
conv_id = uuid.uuid4()
memory_store = ConversationMemory(
    embedder, index_folder=history_index_name, reset_index=True
)
conversation_list = [
    "What is the capital of France?",
    "Tell me more about its history.",
    "List two famous landmarks in that city.",
]
enhanced_queries = process_conversation_and_enhance_query(
    memory_store, conversation_list, perception_chain, conv_id
)
print(enhanced_queries)

2025-07-05 21:02:47.854 | WARNING  | utils:check_and_reset_index:27 - reset_index is set to True. Deleting existing index at 'd:\Study\Programs\TSAI\EAG_V1\Assignment_7_RAG_1\2_URL_RAG\history_index'
2025-07-05 21:02:47.856 | SUCCESS  | utils:check_and_reset_index:32 - Successfully deleted index folder 'd:\Study\Programs\TSAI\EAG_V1\Assignment_7_RAG_1\2_URL_RAG\history_index'


['What is the capital city of France?', 'Tell me more about the history of France, particularly focusing on its capital, Paris.', 'List two famous landmarks in Paris, the capital city of France.']


### Decision and action code

- For the constraints of mcp we cannot use the decision node in jupter notebook , so we have executed the decision node in the test_client.py file and have saved the output in the test_client_output.pkl file
- We will try to debug the action node here



In [3]:
decison_op = "client/test_client_output.json"
import json
with open(decison_op, "r") as f:
    data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'client/test_client_output.json'